In [ ]:
import os
import re
import nltk
import requests
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download('punkt')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import HashingVectorizer

from PIL import Image
warnings.filterwarnings('ignore')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
books = pd.read_csv('/content/Book1.csv')
books.head(3)

,Book,Author,Description,Genres,Thumbnail
0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",http://books.google.com/books/content?id=PGR2A...
1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",http://books.google.com/books/content?id=N8yOE...
2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical...",http://books.google.com/books/content?id=yxIHA...


In [ ]:
df=books.copy()
df = df.drop_duplicates(subset=['Book'])
df.reset_index(drop=True, inplace=True)
df['index'] = [i for i in range(df.shape[0])]
print(df.shape)

(9690, 6)


In [ ]:


filler_words = ['Alternate', 'cover', 'edition', 'ISBN',
                'first','published','story','book','novel','originally',
                'classic','bestselling','bestseller']
summary_filtered = []
genre_filtered = []

for i in df['Description']:

    i = re.sub("[^a-zA-Z]"," ",str(i)).lower()
    i = nltk.word_tokenize(i)
    i = [word for word in i if not word in set(stopwords.words("english"))]
    i = [word for word in i if not word in filler_words]
    i = " ".join(i)
    summary_filtered.append(i)

df['Description'] = summary_filtered

for i in df['Genres']:

    i = re.sub("[^a-zA-Z]"," ",str(i)).lower()
    i = nltk.word_tokenize(i)
    i = " ".join(i)
    genre_filtered.append(i)

df['Genres'] = genre_filtered

target_cols = ['Author','Genres','Description']
df['combined_features'] = df[target_cols].astype(str).apply(lambda row: ' '.join(row.values), axis=1)


In [ ]:
vectorizer = HashingVectorizer()
X = vectorizer.transform(df['combined_features'])

In [ ]:
def recommend_books(book_index):
    similarity_scores = cosine_similarity(X[book_index], X).flatten()
    similar_books = df[['Book', 'Author']].copy()
    similar_books['similarity'] = similarity_scores
    similar_books = similar_books.sort_values(by='similarity', ascending=False)
    print(df.iloc[book_index])
    print("books similar")
    print(similar_books.head(20))

In [ ]:
recommend_books(5)

Book                                                 The Little Prince
Author                                        Antoine de Saint-Exupéry
Description          pilot stranded desert awakes one morning see s...
Genres               classics fiction fantasy childrens france phil...
Thumbnail            http://books.google.com/books/content?id=OPbtD...
index                                                                5
combined_features    Antoine de Saint-Exupéry classics fiction fant...
Name: 5, dtype: object
books similar
                                                   Book  \
5                                     The Little Prince   
3268                                   Rikki-Tikki-Tavi   
259                               The Neverending Story   
2990                                       I'm With You   
3432  The Indian in the Cupboard (The Indian in the ...   
2993                                             Claire   
390      Little House on the Prairie (Little House, #

In [ ]:
import pickle
pickle.dump(X,open('/content/modelX.pkl','wb'))

In [ ]:
pickle.dump(df,open('/content/data.pkl','wb'))